# Trabalho Final - Pesquisa Operacional <br>
## Aplicando Branch-and-Bound


Alunos:<br>
    João Victor Soares Silva, mat.: 20210027300 <br>
    Yhasmim de Souza Tigre, mat.: 20210026966 


Implementação:

In [1]:
#Bibliotecas
from mip import (CBC, CONTINUOUS, MAXIMIZE, Model, xsum, OptimizationStatus)
from pandas import DataFrame

In [2]:
''' FUNÇÃO QUE APLICA ARQUIVO DE TEXTO EM MODELO
           DE PROGRAMAÇÃO LINEAR INTEIRA'''

def modelo(arquivo: str):

    #lendo arquivo de texto
    arquivo = open(arquivo, 'r')

    #lendo as linhas do arquivo
    linhas = arquivo.readlines()
    
    #separando variáveis de restrições
    num_v = int(linhas[0].split()[0])
    num_r = int(linhas[0].split()[1])    

    arquivo.close()

    #criando modelo
    model = Model(sense = MAXIMIZE, solver_name = CBC)

    #criando variáveis
    x = [model.add_var( var_type=CONTINUOUS, lb=0.0, ub=1.0,
         name="x_"+str(i)) for i in range(num_v)]   
        
    #função objetivo    
    obj = [float(i) for i in linhas[1].split()]
    model.objective = xsum(obj[i]*x[i] for i in range(num_v)) 

    #adicionando os coeficientes    
    for c in range (2, num_r+1):
      coefientes = []
      for i in range(num_v + 1):
        coe = float(linhas[c].split()[i])
        coefientes.append(coe)

      #cria as restrições
      model += xsum(coefientes[i]*x[i] for i in range(num_v)) <= coefientes[-1] 

    #diminuindo o tamanho do print
    model.verbose = 0

    return model

In [3]:
''' CLASSE DE CADA NÓ DO ALGORITMO QUE CONTÉM CADA MODELO'''

class No:
    def __init__ (self, modelo : Model):
        self.modelo = modelo

        #otimiza o modelo assim que cria o nó
        self.modelo.optimize()
        
        #salva o valor da função objetivo
        self.Z = self.modelo.objective_value

In [5]:
'''FUNCOES DE PODAGEM DE NÓS'''

#Testa se o nó pode ser podado por limitante:
def podar_limite (lim_primal : float, Z : int):
    if Z <= lim_primal:
        return True
    else:
        return False

#Testa se o nó pode ser podado por integralidade:
def podar_integral (modelo : Model):
    for var in modelo.vars:
        if var.x % 1 != 0:
            return False

    return True   

#Testa se o nó pode ser podado por inviabilidade:
def podar_inviavel (modelo : Model):

    #Inviável:
    if modelo.status == OptimizationStatus.INFEASIBLE:
        return True
    
    #Não encontrou soluções:
    elif modelo.status == OptimizationStatus.NO_SOLUTION_FOUND:
        return True
    
    else:
        return False

In [6]:
'''FUNÇÃO PARA IMPRIMIR O RESULTADO DE UM MODELO'''

def print_resultado (otimo : Model):
    lista_dados = []

    print (f"Resultado da otimização: {otimo.objective_value}\n")
    print ("Valor das Variáveis:")
    
    for var in otimo.vars:
        lista_dados.append([var.name, var.x])
    
    saida = DataFrame(data = lista_dados, columns = ['Variável', 'Valor'])
    print (saida.to_string(index = False))

    return

In [7]:
'''GERA OS DOIS NÓS FILHOS DO ALGORITMO A PARTIR DA VARIÁVEL MAIS PRÓXIMA DE 0,5'''

def passar_no (pai : No):

    dif_abs = 1
    for var in pai.modelo.vars:
        if abs (0.5 - var.x) <= dif_abs:
            dif_abs = abs (0.5 - var.x)
            v_escolhida = var.name

    #cria os modelos filhos
    modelo_0 = pai.modelo.copy()
    modelo_1 = pai.modelo.copy()

    modelo_0.verbose = 0
    modelo_1.verbose = 0

    #nova restrição
    modelo_0 += modelo_0.vars[v_escolhida] == 0
    modelo_1 += modelo_1.vars[v_escolhida] == 1 

    no_0 = No(modelo_0)
    no_1 = No(modelo_1)

    return no_0, no_1

In [15]:
'''CLASSE DO ALGORITMO, COM FUNÇÃO DE EXECUÇÃO:'''

class Branch_and_Bound:
    def __init__ (self, no_modelo : No):
        self.no_modelo = no_modelo
    
    #Executa o algoritmo de Branch and Bound utilizando uma busca em largura:
    def executar_branch_and_bound (self):
        lim_primal = float('-inf')
        fila_modelos = [self.no_modelo]
        solucao = "Não foram encontradas soluções."

        #será interado enquanto houverem nós na fila:
        while len (fila_modelos):
            no_atual = fila_modelos[0] #no

            #Referenciando podagens:

            if podar_inviavel(no_atual.modelo):
                del fila_modelos[0]
                continue
            
            if podar_integral(no_atual.modelo):
                if lim_primal <= no_atual.Z:
                    lim_primal = no_atual.Z
                    solucao = no_atual.modelo
                del fila_modelos[0]
                continue
            
            if podar_limite (lim_primal, no_atual.Z):
                del fila_modelos[0]
                continue
            
            #passa o nó atual para a função que gera os nós filhos:
            no_0, no_1 = passar_no(no_atual)
            fila_modelos.append(no_0)
            fila_modelos.append(no_1)
            del fila_modelos[0]
        
        print_resultado(solucao)

In [16]:
'''TESTES DE EXECUÇÃO'''

teste_1 = No(modelo('./testes/teste1.txt'))
teste_2 = No(modelo('./testes/teste2.txt'))
teste_3 = No(modelo('./testes/teste3.txt'))
teste_4 = No(modelo('./testes/teste4.txt'))

In [22]:
'''INSTANCIANDO O ALGORITMO'''

BaB_1 = Branch_and_Bound(teste_1)
BaB_2 = Branch_and_Bound(teste_2)
BaB_3 = Branch_and_Bound(teste_3)
BaB_4 = Branch_and_Bound(teste_4)

In [23]:
BaB_1.executar_branch_and_bound()

Resultado da otimização: 20.0

Valor das Variáveis:
Variável  Valor
     x_0    0.0
     x_1    1.0
     x_2    0.0
     x_3    0.0
     x_4    1.0
     x_5    0.0
     x_6    0.0


In [24]:
BaB_2.executar_branch_and_bound()

Resultado da otimização: 24.0

Valor das Variáveis:
Variável  Valor
     x_0    0.0
     x_1    1.0
     x_2    0.0
     x_3    0.0
     x_4    0.0
     x_5    1.0
     x_6    1.0
     x_7    0.0
     x_8    0.0


In [25]:
BaB_3.executar_branch_and_bound()

Resultado da otimização: 19.0

Valor das Variáveis:
Variável  Valor
     x_0    0.0
     x_1    0.0
     x_2    1.0
     x_3    0.0
     x_4    0.0
     x_5    0.0
     x_6    1.0
     x_7    0.0
     x_8    0.0


In [26]:
BaB_4.executar_branch_and_bound()

Resultado da otimização: 10.0

Valor das Variáveis:
Variável  Valor
     x_0    0.0
     x_1    0.0
     x_2    1.0
     x_3    0.0
     x_4    0.0
     x_5    0.0
     x_6    0.0
     x_7    0.0
     x_8    0.0
